### HW04: Practice with feature engineering, splitting data, and fitting and regularizing linear models

Alexander Scheibe
avscheibe

### Hello Students:

- Start by downloading HW04.ipynb from this folder. Then develop it into your solution.
- Write code where you see "... your code here ..." below.
  (You are welcome to use more than one cell.)
- If you have questions, please ask them in class or office hours. Our TA
  and I are very happy to help with the programming (provided you start early
  enough, and provided we are not helping so much that we undermine your learning).
- When you are done, run these Notebook commands:
  - Shift-L (once, so that line numbers are visible)
  - Kernel > Restart and Run All (run all cells from scratch)
  - Esc S (save)
  - File > Download as > HTML
- Turn in:
  - HW04.ipynb to Canvas's HW04.ipynb assignment
  - HW04.html to Canvas's HW04.html assignment
  - As a check, download your files from Canvas to a new 'junk' folder. Try 'Kernel > Restart
  and Run All' on the '.ipynb' file to make sure it works. Glance through the '.html' file.
- Turn in partial solutions to Canvas before the deadline. e.g. Turn in part 1,
  then parts 1 and 2, then your whole solution. That way we can award partial credit
  even if you miss the deadline. We will grade your last submission before the deadline.

In [51]:
# ... your code here ... (import statements)
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error


## 1. Feature engineering (one-hot encoding and data imputation)

(Note: This paragraph is not instructions but rather is to communicate context for this exercise. We use the same Titanic data we used in HW02:
- There we used `df.dropna()` to drop any observations with missing values; here we use data imputation instead.
- There we manually did one-hot encoding of the categorical `Sex` column by making a `Female` column; here we do the same one-hot encoding with the help of pandas's `df.join(pd.get_dummies())`.
- There we used a decision tree; here we use $k$-NN.

We evaluate how these strategies can improve model performance by allowing us to use columns with categorical or missing data.)

### 1a. Read the data from [http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv](http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv).
- Retain only these columns: Survived, Pclass, Sex, Age, SibSp, Parch.
- Display the first 7 rows.

These data are described at [https://www.kaggle.com/competitions/titanic/data](https://www.kaggle.com/competitions/titanic/data) (click on the small down-arrow to see the "Data Dictionary"), which is where they are from.
- Read that "Data Dictionary" paragraph (with your eyes, not python) so you understand what each column represents.


In [52]:
data = pd.read_csv("kaggle_titanic_train.csv")

# Retain only the specified columns
columns_to_keep = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
data = data[columns_to_keep]

# Display the first 7 rows
data.head(7)

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0
5,0,3,male,NaN,0,0
6,0,1,male,54.0,0,0


### 1b. Try to train a $k$NN model to predict $y=$ 'Survived' from $X=$ these features: 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch'.
- Use $k = 3$ and the (default) euclidean metric.
- Notice at the bottom of the error message that it fails with the error "ValueError: could not convert string to float: 'male'".
- Comment out your .fit() line so the cell can run without error.

In [53]:
from sklearn.neighbors import KNeighborsClassifier

columns_to_keep = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
data = data[columns_to_keep]

X = data.drop('Survived', axis=1)
y = data['Survived']

knn = KNeighborsClassifier(n_neighbors=3)

## knn.fit(X, y)

### 1c. Try to train again, this time without the 'Sex' feature.
- Notice that it fails because "Input contains NaN".
- Comment out your .fit() line so the cell can run without error.
- Run `X.isna().any()` (where X is the name of your DataFrame of features) to see that
  the 'Age' feature has missing values. (You can see the first missing value in
  the sixth row that you displayed above.)

In [54]:
columns_to_keep = ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch']
data = data[columns_to_keep]

X = data.drop('Survived', axis=1)
y = data['Survived']

knn = KNeighborsClassifier(n_neighbors=3)

# knn.fit(X, y)

X.isna().any()

Pclass    False
Age        True
SibSp     False
Parch     False
dtype: bool

### 1d. Train without the 'Sex' and 'Age' features.
- Report accuracy on the training data with a line of the form
  `Accuracy on training data is  0.500` (0.500 may not be correct).

In [55]:
# ... your code here ...
columns_to_keep = ['Survived', 'Pclass', 'SibSp', 'Parch']
data = data[columns_to_keep]

X = data.drop('Survived', axis=1)
y = data['Survived']

# Initialize the kNN model with k=3 and Euclidean metric
knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X, y)

y_pred = knn.predict(X)

accuracy = accuracy_score(y, y_pred)

print(f"Accuracy on training data is {accuracy:.3f}")

Accuracy on training data is 0.633


### 1e.  Use one-hot encoding
to include a binary 'male'  feature made from the 'Sex' feature. (Or include a binary 'female'
feature, according to your preference. Using both is unnecessary since either is the logical
negation of the other.) That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male'.
- Use pandas's df.join(pd.get_dummies())`.
- Report training accuracy as before.

In [56]:
# ... your code here ...
data = pd.read_csv("kaggle_titanic_train.csv")

columns_to_keep = ['Survived', 'Pclass', 'SibSp', 'Parch', 'Sex']
data = data[columns_to_keep]

data = pd.get_dummies(data, columns=['Sex'], drop_first=True)

X = data.drop('Survived', axis=1)
y = data['Survived']

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X, y)

y_pred = knn.predict(X)

accuracy = accuracy_score(y, y_pred)

print(f"Accuracy on training data is {accuracy:.3f}")


Accuracy on training data is 0.744


### 1f. Use data imputation
to include an 'age' feature made from 'Age' but replacing each missing value with the median
of the non-missing ages. That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male',
'age'.

- Report training accuracy as before.

In [57]:
data = pd.read_csv("kaggle_titanic_train.csv")

columns_to_keep = ['Survived', 'Pclass', 'SibSp', 'Parch', 'Sex', 'Age']
data = data[columns_to_keep]

data = pd.get_dummies(data, columns=['Sex'], drop_first=True)

imputer = SimpleImputer(strategy='median')
data['Age'] = imputer.fit_transform(data[['Age']])

X = data.drop('Survived', axis=1)
y = data['Survived']

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X, y)

y_pred = knn.predict(X)

accuracy = accuracy_score(y, y_pred)

print(f"Accuracy on training data is {accuracy:.3f}")

Accuracy on training data is 0.861


## 2. Explore model fit, overfit, and regularization in the context of multiple linear regression

### 2a. Prepare the data:
- Read [http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv](http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv) into a DataFrame.
- Set a variable `X` to the subset consisting of all columns except `mpg`.
- Set a variable `y` to the `mpg` column.
- Use `train_test_split()` to split `X` and `y` into `X_train`, `X_test`, `y_train`, and `y_test`.
  - Reserve half the data for training and half for testing.
  - Use `random_state=0` to get reproducible results.

In [64]:
data = pd.read_csv("mtcars.csv")

X = data.drop('mpg', axis=1)
X = pd.get_dummies(X, columns=['Unnamed: 0'], drop_first=True)

y = data['mpg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,...,Unnamed: 0_Merc 280C,Unnamed: 0_Merc 450SE,Unnamed: 0_Merc 450SL,Unnamed: 0_Merc 450SLC,Unnamed: 0_Pontiac Firebird,Unnamed: 0_Porsche 914-2,Unnamed: 0_Toyota Corolla,Unnamed: 0_Toyota Corona,Unnamed: 0_Valiant,Unnamed: 0_Volvo 142E
0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,...,0,0,0,0,0,0,0,0,0,0
1,6,160.0,110,3.90,2.875,17.02,0,1,4,4,...,0,0,0,0,0,0,0,0,0,0
2,4,108.0,93,3.85,2.320,18.61,1,1,4,1,...,0,0,0,0,0,0,0,0,0,0
3,6,258.0,110,3.08,3.215,19.44,1,0,3,1,...,0,0,0,0,0,0,0,0,0,0
4,8,360.0,175,3.15,3.440,17.02,0,0,3,2,...,0,0,0,0,0,0,0,0,0,0
5,6,225.0,105,2.76,3.460,20.22,1,0,3,1,...,0,0,0,0,0,0,0,0,1,0
6,8,360.0,245,3.21,3.570,15.84,0,0,3,4,...,0,0,0,0,0,0,0,0,0,0
7,4,146.7,62,3.69,3.190,20.00,1,0,4,2,...,0,0,0,0,0,0,0,0,0,0
8,4,140.8,95,3.92,3.150,22.90,1,0,4,2,...,0,0,0,0,0,0,0,0,0,0
9,6,167.6,123,3.92,3.440,18.30,1,0,4,4,...,0,0,0,0,0,0,0,0,0,0


### 2b. Train three models on the training data and evaluate each on the test data:
- `LinearRegression()`
- `Lasso()`
- `Ridge()`

The evaluation consists in displaying MSE$_\text{train}, $ MSE$_\text{test}$, and the coefficients $\mathbf{w}$ for each model.

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)

ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)

y_train_pred_linear = linear_reg.predict(X_train)
y_test_pred_linear = linear_reg.predict(X_test)

y_train_pred_lasso = lasso_reg.predict(X_train)
y_test_pred_lasso = lasso_reg.predict(X_test)

y_train_pred_ridge = ridge_reg.predict(X_train)
y_test_pred_ridge = ridge_reg.predict(X_test)

mse_train_linear = mean_squared_error(y_train, y_train_pred_linear)
mse_test_linear = mean_squared_error(y_test, y_test_pred_linear)

mse_train_lasso = mean_squared_error(y_train, y_train_pred_lasso)
mse_test_lasso = mean_squared_error(y_test, y_test_pred_lasso)

mse_train_ridge = mean_squared_error(y_train, y_train_pred_ridge)
mse_test_ridge = mean_squared_error(y_test, y_test_pred_ridge)

print("Linear Regression:")
print(f"MSE_train: {mse_train_linear}")
print(f"MSE_test: {mse_test_linear:.3f}")
print("Coefficients: ", linear_reg.coef_)

ValueError: could not convert string to float: 'Camaro Z28'

### 2c. Answer a few questions about the models:
- Which one best fits the training data?
- Which one best fits the test data?
- Which one does feature selection by setting most coefficients to zero?

1. Linear Regression as it has the lowest MSE across all 3 options
2. Ridge regression as it has the lowest MSE on the test data across all 3 options.
3. Lass regression set most coeeficients to 0 in feature selection.